# Training Tensorflow model on Google Cloud AI Platform

In [2]:
!sudo chown -R jupyter:jupyter /home/jupyter/Cancer-Detection-using-GCP

In [3]:
import os

In [9]:
PROJECT = 'vernal-buffer-285906'
REGION = 'us-west1'
BUCKET = 'vernal-buffer-285906'

In [10]:
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.6'

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## Submit Training job to Google Cloud Ai Platform

In [12]:
%%bash

# Move latest config file to google storage bucket
gsutil -m cp -r ../../config gs://$BUCKET

# Set PYTHONPATH enviornment variable
export PYTHONPATH=${PYTHONPATH}:/home/jupyter/Cancer-Detection-using-GCP/

# Specify a unique name for the job.
JOBNAME=cancer_detector_$(date -u +%y%m%d_%H%M%S)
echo $REGION $JOBNAME

# Submit training job
gcloud ai-platform jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=detectors.tf_gcp.trainer.task \
  --package-path=/home/jupyter/Cancer-Detection-using-GCP/detectors/ \
  --job-dir=gs://$BUCKET \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=BASIC_GPU \
  --runtime-version=2.5 \
  --python-version=3.7 \
  -- \
  --train-config=gs://$BUCKET/config/config.yaml

us-west1 cancer_detector_210911_060551
jobId: cancer_detector_210911_060551
state: QUEUED


Copying file://../../config/config.yaml [Content-Type=application/octet-stream]...
Copying file://../../config/.ipynb_checkpoints/config-checkpoint.yaml [Content-Type=application/octet-stream]...
/ [2/2 files][  3.4 KiB/  3.4 KiB] 100% Done                                    
Operation completed over 2 objects/3.4 KiB.                                      
Job [cancer_detector_210911_060551] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe cancer_detector_210911_060551

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs cancer_detector_210911_060551


In [ ]:
# Running TensorBoard
# Open the port you are supposed to use (usually 6006) in Google cloud vm instance
# info -> https://docs.bitnami.com/google/faq/administration/use-firewall/
# Run tensorboard using following command
# tensorboard --logdir <path/to/log/directory> --bind_all
# Ex:
# tensorboard --logdir gs://vernal-buffer-285906/train_results/tensorboard --bind_all